In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq ollama==0.5.1 --progress-bar off

In [3]:
!gdown -qqq 1DhAkbU-a8k8wx5ckoJHzfOLNtKc0sU9_ -O data/

In [ ]:
import textwrap
from enum import Enum
from pathlib import Path

import ollama
from ollama import GenerateResponse

MODEL = "deepseek-r1:8b"
TEMPERATURE = 0.6
TOP_P = 0.95
CONTEXT_WINDOW = 8192

nvidia_earnings = Path("data/nvidia-april-25.md").read_text()

In [2]:
print(nvidia_earnings[:1000])

NVIDIA Announces Financial Results for First Quarter Fiscal 2026

- Revenue of $44.1 billion, up 12% from Q4 and up 69% from a year ago
- Data Center revenue of $39.1 billion, up 10% from Q4 and up 73% from a year ago

NVIDIA (NASDAQ: NVDA) today reported revenue for the first quarter ended April 27, 2025, of $44.1 billion, up 12% from the previous quarter and up 69% from a year ago.

On April 9, 2025, NVIDIA was informed by the U.S. government that a license is required for exports of its H20 products into the China market. As a result of these new requirements, NVIDIA incurred a $4.5 billion charge in the first quarter of fiscal 2026 associated with H20 excess inventory and purchase obligations as the demand for H20 diminished. Sales of H20 products were $4.6 billion for the first quarter of fiscal 2026 prior to the new export licensing requirements. NVIDIA was unable to ship an additional $2.5 billion of H20 revenue in the first quarter.

For the quarter, GAAP and non-GAAP gross mar

## Ollama Setup


In [3]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"


def call_model(
    prompt: str,
    allow_thinking: bool = True,
    response_format: ResponseFormat = ResponseFormat.TEXT,
) -> GenerateResponse:
    return ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        think=allow_thinking,
        format="" if response_format == ResponseFormat.TEXT else "json",
        options={
            "temperature": TEMPERATURE,
            "top_p": TOP_P,
            "num_ctx": CONTEXT_WINDOW,
        },
    )

In [4]:
%%time
response = call_model("Hello")

CPU times: user 4.76 ms, sys: 14 ms, total: 18.7 ms
Wall time: 17.6 s


In [5]:
def print_text(text: str):
    text = [textwrap.fill(line, width=100) for line in text.split("\n")]
    print("\n".join(text))

In [6]:
print_text(response.thinking)

Okay, the user just said “Hello” with a simple lowercase. Hmm, seems like they're initiating a
casual conversation or testing how I respond to basic greetings.

First impression: this is probably an opening move - maybe they're unsure what to ask yet, or
checking if I'm awake/online. The tone feels neutral but friendly since it's not formal “Greetings”
or anything aggressive.

Given the minimal input, my response should be warm and open-ended to encourage them to share more.
No need for deep analysis here unless they follow up with complex questions later.

I'll match their casual tone while keeping it helpful - using a smile emoji feels appropriate since
text lacks vocal tones. The invitation (“What can I help you with?”) leaves the door wide open
without being pushy.

Potential scenarios:
- They might be lonely and just wants someone to talk
- Could be a developer testing API response
- Might have accidentally triggered chatbot

Since they didn't specify gender or name, keeping it ne

In [7]:
print_text(response.response)

Hello! 😊 How can I assist you today?


### Enable/Disable Thinking


In [8]:
%%time
response = call_model("Hello", allow_thinking=False)

CPU times: user 4.55 ms, sys: 7.77 ms, total: 12.3 ms
Wall time: 2.17 s


In [9]:
print(response.thinking)

None


In [10]:
print_text(response.response)

Hello! 👋 How are you today? Nice to meet you. I'm an AI assistant here to help answer your
questions, provide information, or just chat if you'd like. Let me know what's on your mind!


## Document Analysis


In [11]:
PROMPT = textwrap.dedent("""
You're an expert financial analyst. You're analyzing financial reports and earnings calls.

Your goal is to accurately answer questions about the document using the information provided in the text.

<instructions>
- Carefully consider the complete content of the document
- If the answer is not explicitly stated, always state that the information is not available in the document
- Provide answers in a concise and clear manner
- Give contextual information when necessary, but avoid unnecessary details
</instructions>

<output_format
Always reply with a Markdown that contains properly formatted elements - headings, lists, tables etc.
</output_format>

<text>
{document}
</text>

<question>
{question}
</question>

Answer the question in Markdown format based on the document above. If the answer is not available, state that the information is not available in the document.

Your answer:
""").strip()

### Summarization


In [12]:
question = "Provide a concise summary of the earnings call, including key financial metrics and any significant developments mentioned."
prompt = PROMPT.format(document=nvidia_earnings, question=question)

In [13]:
%%time
response = call_model(prompt)

CPU times: user 6.32 ms, sys: 36.9 ms, total: 43.2 ms
Wall time: 1min 38s


In [14]:
print_text(response.thinking)

Hmm, the user wants me to provide a concise summary of an NVIDIA earnings call including key
financial metrics and significant developments. They've provided extensive financial data from their
reports, so I need to carefully analyze this information to extract what's most important.

Looking at this document, it contains detailed financial results for NVIDIA's first quarter of
fiscal 2026. The revenue was $44.1 billion, which is a 12% increase from the previous quarter and a
69% increase year-over-year. Data Center revenue specifically was $39.1 billion with similar growth
patterns.

There were some significant developments mentioned - particularly regarding export controls for H20
products to China, resulting in a $4.5 billion charge due to excess inventory and purchase
obligations. This seems like an important point that explains why some of the previous quarter's
revenue wasn't realized this time around.

The company also discussed their outlook for the second quarter with expected

In [15]:
print_text(response.response)

**Earnings Call Summary: NVIDIA Q1 Fiscal 2026**

NVIDIA reported strong first-quarter financial results for fiscal year 2026, driven by continued
growth in its Data Center segment. Key highlights include:

- **Revenue**: $44.1 billion (Q1 FY26), representing a 12% quarter-over-quarter increase and a
significant 69% year-over-year jump.
- **Data Center Revenue**: $39.1 billion, the largest growth driver for this quarter.

The company also incurred a substantial charge ($4.5B) related to H20 products due to new U.S.
export controls limiting sales into China. This impacted revenue projections and margins but was
addressed in the non-GAAP metrics provided.

Notable developments discussed include:
- The launch of the **Blackwell NVL72 AI supercomputer**, positioned as a key infrastructure for
agentic AI.
- Major partnerships such as with **HUMAIN** (Saudi Arabia), G42, Oracle, and SoftBank Group to
advance AI computing globally.
- Expansion into new markets via projects like **Stargate UAE

**Earnings Call Summary: NVIDIA Q1 Fiscal 2026**

NVIDIA reported strong first-quarter financial results for fiscal year 2026, driven by continued
growth in its Data Center segment. Key highlights include:

- **Revenue**: $44.1 billion (Q1 FY26), representing a 12% quarter-over-quarter increase and a
  significant 69% year-over-year jump.
- **Data Center Revenue**: $39.1 billion, the largest growth driver for this quarter.

The company also incurred a substantial charge ($4.5B) related to H20 products due to new U.S.
export controls limiting sales into China. This impacted revenue projections and margins but was
addressed in the non-GAAP metrics provided.

Notable developments discussed include:

- The launch of the **Blackwell NVL72 AI supercomputer**, positioned as a key infrastructure for
  agentic AI.
- Major partnerships such as with **HUMAIN** (Saudi Arabia), G42, Oracle, and SoftBank Group to
  advance AI computing globally.
- Expansion into new markets via projects like **Stargate UAE** in Abu Dhabi and plans to
  collaborate with Foxconn on an AI factory supercomputer.

NVIDIA's outlook for Q2 FY26 anticipates revenue around $45 billion, reflecting adjustments due to
H20 limitations. The company remains focused on scaling its AI infrastructure capabilities through
innovation and partnerships.


### P&L Analysis


In [17]:
%%time

question = """
Extract P&L summary for the Q1 Fiscal 2026 (GAAP):

- Revenue
- Gross Margin
- Operating Income
- Net Income
- Dilluted Earnings Per Share (EPS)
"""
prompt = PROMPT.format(document=nvidia_earnings, question=question)
response = call_model(prompt)

CPU times: user 5.27 ms, sys: 25.5 ms, total: 30.7 ms
Wall time: 48.5 s


In [18]:
print_text(response.thinking)

Okay, let's tackle this query step by step. The user wants a P&L summary for Q1 Fiscal 2026 under
GAAP, specifically focusing on Revenue, Gross Margin, Operating Income, Net Income, and Diluted EPS.

First, I'll scan through the provided text to find all relevant financial figures in GAAP terms.

The earnings press release gives some key numbers: Revenue is $44.1 billion (or $44,062 million). It
mentions a 12% quarter-over-quarter increase and a 69% year-over-year growth from the prior year's
Q1.

Next, Gross Margin comes up as 60.5%. However, there's also mention of excluding charges related to
H20 products, which affects this number. But since we're focusing on GAAP, I'll stick with the
reported figure here.

For Operating Income, the press release states $21,638 million for Q1 FY26 under GAAP.

Net Income is clearly stated as $18,775 million in the GAAP section of the income statement table
provided later.

Diluted EPS (Diluted earnings per share) appears twice: once at 0.76% change

In [20]:
print_text(response.response)

## P&L Summary for Q1 Fiscal 2026 (GAAP)

Based on the provided document:

| Metric                           | Amount      |
| :------------------------------- | :---------- |
| **Revenue**                        | $44,062 million |
| **Gross Margin**                   | 60.5%        |
| **Operating Income**               | $21,638 million |
| **Net Income**                      | $18,775 million |
| **Diluted Earnings Per Share (EPS)** | $0.76        |

This information is explicitly stated in the earnings call transcript and financial tables provided
within the document.


## P&L Summary for Q1 Fiscal 2026 (GAAP)

Based on the provided document:

| Metric                               | Amount          |
| :----------------------------------- | :-------------- |
| **Revenue**                          | $44,062 million |
| **Gross Margin**                     | 60.5%           |
| **Operating Income**                 | $21,638 million |
| **Net Income**                       | $18,775 million |
| **Diluted Earnings Per Share (EPS)** | $0.76           |

This information is explicitly stated in the earnings call transcript and financial tables provided
within the document.


True values:

NVIDIA's Q1 Fiscal 2026 P&L (GAAP), figures are for the quarter ended April 27, 2025:

1.  **Revenue:** $44.1 billion
2.  **GAAP Gross Margin:** 60.5%
3.  **GAAP Operating Income:** $21.6 billion
4.  **GAAP Net Income:** $18.8 billion
5.  **GAAP Diluted Earnings Per Share (EPS):** $0.76


## Balance Sheet


In [25]:
%%time

question = """
Extract information about the Balance Sheet for the Q1 Fiscal 2026 (GAAP):

- Cash, cash equivalents and marketable securities
- Inventories
- Accounts receivable, net
- Total Liabilities
- Total Shareholders' Equity
"""
prompt = PROMPT.format(document=nvidia_earnings, question=question)
response = call_model(prompt)

CPU times: user 6.09 ms, sys: 14.9 ms, total: 21 ms
Wall time: 38.4 s


In [26]:
print_text(response.thinking)

Okay, let's tackle this query as a financial analyst would. The user wants specific balance sheet
figures for NVIDIA's Q1 Fiscal 2026 under GAAP.

First, I need to carefully scan the provided text for relevant financial data. Looking at the
Condensed Consolidated Balance Sheets section labeled "April 27, 2025" – this is clearly Q1 FY26
since dates match April 27 being recent and January 26 representing prior year comparison.

For Cash: The GAAP line shows $53,691 million at the end of period (April 27, 2025), which I'll
confirm matches the "Cash and cash equivalents" amount in the Statement of Cash Flows.

Inventories are straightforward – they're listed as current assets with a GAAP value of $11,333
million on April 27.

Accounts receivable appears twice: In the Balance Sheet it's netted at $22,132 mil, and in the
reconciliation table showing pre-split comparability. Both confirm consistency.

Total Liabilities require summing current ($41,408) + long-term ($5,964) but wait – actually

In [27]:
print_text(response.response)

# NVIDIA Q1 Fiscal 2026 Balance Sheet (GAAP)

Based on the provided text:

## Assets

- **Cash, cash equivalents and marketable securities:** $53,691 million
- **Inventories:** $11,333 million
- **Accounts receivable, net:** $22,132 million

## Liabilities & Shareholders' Equity

- **Total liabilities:** $41,411 million (includes all current and long-term liabilities)
- **Shareholders' equity:** $83,843 million


# NVIDIA Q1 Fiscal 2026 Balance Sheet (GAAP)

Based on the provided text:

## Assets

- **Cash, cash equivalents and marketable securities:** $53,691 million
- **Inventories:** $11,333 million
- **Accounts receivable, net:** $22,132 million

## Liabilities & Shareholders' Equity

- **Total liabilities:** $41,411 million (includes all current and long-term liabilities)
- **Shareholders' equity:** $83,843 million


True values:

1.  **Cash, cash equivalents and marketable securities:** $53,691 million
2.  **Inventories:** $11,333 million
3.  **Accounts receivable, net:** $22,132 million
4.  **Total Liabilities:** $41,411 million
5.  **Total Shareholders' Equity:** $83,843 million


## Cashflow Statement


In [22]:
%%time

question = """
Extract information about the Cashflow Statement for the Q1 Fiscal 2026 (GAAP):

- Net cash provided by operating activities
- Free cash flow
- Purchases of property and equipment
- Payments related to purchases of common stock
- Cash and cash equivalents at end of period
"""
prompt = PROMPT.format(document=nvidia_earnings, question=question)
response = call_model(prompt)

CPU times: user 4.83 ms, sys: 32 ms, total: 36.9 ms
Wall time: 55.9 s


In [23]:
print_text(response.thinking)

Hmm, the user wants me to extract specific financial data from NVIDIA's Q1 FY2026 cash flow
statement (GAAP). They're asking for net cash provided by operating activities, free cash flow,
purchases of property and equipment, common stock repurchases, and ending cash equivalents. Let me
carefully review the document.

Looking at the GAAP Cash Flows From Operating Activities section in their condensed income
statements, I see that Net cash provided by operating activities is clearly stated as $27,414
million for Q1 FY2026 (ended April 27, 2025). The beginning balance of cash and cash equivalents was
also available at the start of the period.

For free cash flow, it's calculated in a reconciliation section. I need to find that specific
calculation. Yes, there's a table showing Free cash flow calculations under GAAP net cash provided
by operating activities minus purchases related to property and equipment and intangible assets plus
principal payments on those assets. That comes out to $26

In [24]:
print_text(response.response)

Based on the provided text:

**GAAP Cash Flow Statement for Q1 Fiscal 2026**

| Metric                                      | Amount (in millions) |
| -------------------------------------------- | --------------------:|
| Net cash provided by operating activities    | $27,414               |
| Free cash flow                                | $26,135              |
| Purchase related to property and equipment   | ($1,227)             |
| Payments related to repurchases of common stock | ($14,095)            |
| Cash and cash equivalents at end of period  | $15,234              |

**Note:** The table shows the GAAP figures. Percentages are not provided in the document for these
specific metrics.


Based on the provided text:

**GAAP Cash Flow Statement for Q1 Fiscal 2026**

| Metric                                          | Amount (in millions) |
| ----------------------------------------------- | -------------------: |
| Net cash provided by operating activities       |              $27,414 |
| Free cash flow                                  |              $26,135 |
| Purchase related to property and equipment      |             ($1,227) |
| Payments related to repurchases of common stock |            ($14,095) |
| Cash and cash equivalents at end of period      |              $15,234 |

**Note:** The table shows the GAAP figures. Percentages are not provided in the document for these
specific metrics.


True values:

1.  **Net cash provided by operating activities:** $27,414 million
2.  **Free Cash Flow (FCF):** $26,135 million
3.  **Purchase related to property and equipment and intangible assets (Capital Expenditures):** $1,227 million
4.  **Payments related to repurchases of common stock:** $14,095 million\*\*
5.  **Cash and cash equivalents at end of period:** $15,234 million


## References

- [DeepSeek-R1-0528: How to Run Locally](https://docs.unsloth.ai/basics/deepseek-r1-0528-how-to-run-locally)
